<a href="https://colab.research.google.com/github/hiris25/TEVC-analysis-scripts/blob/master/TEVC_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TEVC - Dose Response**

## Set up libraries

In [1]:
import pandas as pd
import numpy as np
import glob as glob

## Import and prepare data

### Import Files

Import all files from specified folder and combine into one dataframe called 'df'

In [ ]:
path = r'/yourfolder' # use your path
all_files = glob.glob(path + "/*extension.dat") # use your extension

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t') # for .dat files you must specify separation for example sep='\t'
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

### Remove excess coloumns and clean data
Here we are keeping only the relevant information, you can add or remove coloumns as you like.
We will also be standardising the units and generally cleaning up the data

In [ ]:
#coloumn names must be updated to those in your files, these are based on the standard coloumn titles from Robocyte2+ software export
data =  df[['Well'] + ['Buffer'] + ['Comp. 1'] + ['conc. 1'] + ['unit 1'] + ['Minimum'] + ['Baseline Average'] + ['Start Date']]

data['date'] = data['Start Date'].str.extract('(../../....)', expand=True)

Transform units all to uM scale

In [ ]:
#Transform units all to µM scale
conditions = [
    (data['unit 1'] == 'mM'),
    (data['unit 1'] == 'µM'),
    (data['unit 1'] == 'nM')]

choices = ['1000', '1', '0.001']

data['factor'] = np.select(conditions, choices, default='0')
data['factor']=data.factor.astype(float)
data['Agonist_Concentration_µM'] = data['conc. 1'] * data['factor']        

In [ ]:
data =  data[['Well'] + ['Buffer'] + ['Comp. 1'] + ['Minimum'] + ['Agonist_Concentration_µM'] + ['date']]
data.reset_index(drop=True)

In [ ]:
#data['log_Agonist_µM'] = np.log(data['Agonist_Concentration_µM']) #if you wish to calculate log concentration
data['Agonist'] = data['Comp. 1']
data['Response'] = data['Minimum']
data = data.drop(columns=['Comp. 1'])

## Transform data to I/Imax for each oocyte

In [ ]:
norm = data.Minimum / data.groupby(['Well']).Minimum.transform(np.min)
data['Response'] = norm
normdata = data.drop(columns=['Minimum', 'Buffer']) #drop unwanted coloumns
normgroup = normdata.groupby(['Agonist_Concentration_µM']).describe()

In [ ]:
grouped_data = normdata.groupby(['Agonist', 'Agonist_Concentration_µM']) #view grouped data
grouped_data['Response'].describe()

## Export as .csv
comment out if you do not want to export .csv
change path and file name before running

exports grouped mean, std, N into a new .csv file

Includes Agonist concentrations in uM

In [ ]:
#export = normdata #select data to export
export = normdata.groupby(['Agonist', 'Agonist_Concentration_µM']) #select groups to average
export = export['Response'].describe()

#export = export.loc[export['Agonist_Concentration_µM'] == 1000.0] #if you wish to export a single dose
#export.head(15) #if you wish to view the exported data

export.to_csv(r'/yourpath/filename.csv')